In [1]:
# [+] SparkSession 설정
import findspark
findspark.init()

from pyspark.sql import SparkSession

# [+] SparkSession 객체 생성 및 설정
spark = SparkSession.builder.master('local').appName('mllib-turorial').getOrCreate()

## MLlib 예제: Logistic Regression
+ ```pyspark.ml.linalg```: 선형대수(linear algebra) 관련 패키지
    + ```Vectors```: 벡터 데이터 타입
+ ```pyspark.ml.classification```: 분류 모델 관련 패키지
    + ```LogisticRegression```: 가장 기본적인 분류 모델

In [2]:
# Vectors, LogisticRegression 임포트
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

In [10]:
"""
    데이터 정의: 레이블과 특징
    - dense(): 모든 값을 저장하는 벡터 생성
    - sparse(): 0이 아닌 값만 저장하는 벡터 생성
"""

train_data = [
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),     # (레이블(정답), 특징값(feature))
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5])),
]

In [18]:
# [+] 스키마 정의
schema = ['label', 'features']

In [19]:
# [+] DataFrame 생성
train_df = spark.createDataFrame(train_data, schema)

In [20]:
# [+] DataFrame 출력
train_df.show()

+-----+--------------+
|label|      features|
+-----+--------------+
|  1.0| [0.0,1.1,0.1]|
|  0.0|[2.0,1.0,-1.0]|
|  0.0| [2.0,1.3,1.0]|
|  1.0|[0.0,1.2,-0.5]|
+-----+--------------+



In [21]:
"""
    LogisticRegression (Estimator) 생성
    - maxIter: 최대 학습 횟수 매개변수
    - regParam: 정규화 매개변수
"""

lr = LogisticRegression(maxIter=30, regParam=0.01)

In [22]:
# 모델 학습
model = lr.fit(train_df)

In [23]:
model

LogisticRegressionModel: uid=LogisticRegression_fd54c0f51c70, numClasses=2, numFeatures=3

In [24]:
# 모델 테스트
test_data = [
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5])),
]

In [25]:
# [+] DataFrame 생성
test_df = spark.createDataFrame(test_data, schema)

In [26]:
# 모델 예측
predictions = model.transform(test_df)

In [27]:
# 예측결과 출력
predictions.show()

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0|[-1.0,1.5,1.3]|[-6.2435550918400...|[0.00193916823498...|       1.0|
|  0.0|[3.0,2.0,-0.1]|[5.45228608726759...|[0.99573180142693...|       0.0|
|  1.0|[0.0,2.2,-1.5]|[-4.4104172202339...|[0.01200425500655...|       1.0|
+-----+--------------+--------------------+--------------------+----------+



## LogisticRegression 파이프라인 구현하기
+ 예측 문제: Spark에 대한 텍스트 인지 아닌지를 분류
+ ```Pipeline```: 머신러닝 과정에 대한 파이프라인 정의
+ ```HashingTF```: 컬럼을 용어빈도(term-frequency) 컬럼으로 변환하는 Transformer
+ ```Tokenizer```: 컬럼의 텍스트를 여러 개의 단어로 분할하는 Transformer

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer

In [4]:
# 훈련 데이터에 대한 DataFrame 생성
train_df = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),   # true 1
    (1, "b d", 0.0),               # false 0
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])    # schema 바로 작성

In [5]:
train_df.show()

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



In [6]:
"""
    머신러닝 파이프라인 과정: Tokenizer -> HashingTF -> LinearRegression
"""

# Tokenizer 객체 생성
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [9]:
tokenizer.transform(train_df).show()

+---+----------------+-----+--------------------+
| id|            text|label|               words|
+---+----------------+-----+--------------------+
|  0| a b c d e spark|  1.0|[a, b, c, d, e, s...|
|  1|             b d|  0.0|              [b, d]|
|  2|     spark f g h|  1.0|    [spark, f, g, h]|
|  3|hadoop mapreduce|  0.0| [hadoop, mapreduce]|
+---+----------------+-----+--------------------+



In [10]:
# HashingTF 객체 생성
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [11]:
"""
    [+] LogisticRegression 객체 생성
    - maxIter = 10
    - regParam = 0.001
"""

lr = LogisticRegression(maxIter = 30, regParam = 0.01)

In [13]:
# [+] Pipeline 객체 생성
pipeline = Pipeline(stages = [tokenizer, hashingTF, lr])

In [14]:
# [+] Pipeline 실행 -> 모델 생성
model = pipeline.fit(train_df)

In [15]:
# 테스트 데이터에 대한 DataFrame 생성
test_df = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [16]:
# [+] 테스트 데이터에 대한 예측
predictions = model.transform(test_df)

In [17]:
predictions.show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[19036,68...|[0.37446592544992...|[0.59253766108921...|       0.0|
|  5|             l m n|           [l, m, n]|(262144,[1303,526...|[2.83313663883221...|[0.94444041965042...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[173558,1...|[-1.1622095936644...|[0.23826602233961...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[68303,19...|[3.75513183310007...|[0.97713755512847...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+

